In [1]:
!nvidia-smi

Sun May  7 06:25:58 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   30C    P0    45W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install facenet_pytorch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 52.8 MB/s eta 0:00:00


In [4]:
#importing the libraries
import torch
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
from torch.nn import functional as F
from torch import optim
from torch.utils.data import DataLoader
from torchvision import transforms, utils, datasets, models


In [5]:
from facenet_pytorch import InceptionResnetV1

In [ ]:
help(InceptionResnetV1)

In [6]:
import glob
import cv2
from google.colab.patches import cv2_imshow

In [7]:
import time
import copy

In [8]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cuda:0


**Load data**

train data

In [11]:
data_transform = transforms.Compose([
      transforms.ToTensor(),
      transforms.Resize(224),
      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
  ])

In [12]:
# select the path
target_path = "/content/drive/MyDrive/DLpj/train_target_2_/*.*"
train_data_charm_zu = []
for file in glob.glob(target_path):
  image_read = cv2.imread(file)
  # resize_img = cv2.resize(image_read, (224, 224))
  # conversion numpy array into rgb image to show
  train_data_charm_zu.append(image_read)
  # # wait for 1 second
  # k = cv2.waitKey(1000)
  # # destroy the window
  # cv2.destroyAllWindows()

In [13]:
len(train_data_charm_zu)

1342

In [14]:
train_data_c = []
for img in train_data_charm_zu:
  img = data_transform(img)
  train_data_c.append((img, 1))

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


In [15]:
len(train_data_c)

1342

In [16]:
import random
train_data_c = random.sample(train_data_c, 1044)

In [17]:
len(train_data_c)

1044

In [18]:
# select the path
target_path = "/content/drive/MyDrive/DLpj/train_celebrities/*.*"
train_data_celebrities = []
for file in glob.glob(target_path):
  image_read = cv2.imread(file)
  # resize_img = cv2.resize(image_read, (224, 224))
  train_data_celebrities.append(image_read)
  # # wait for 1 second
  # k = cv2.waitKey(1000)
  # # destroy the window
  # cv2.destroyAllWindows()


In [19]:
len(train_data_celebrities)

1044

In [20]:
train_data_cel = []
for img in train_data_celebrities:
  img = data_transform(img)
  train_data_cel.append((img, 0))

In [21]:
len(train_data_cel)

1044

In [22]:
train_data = train_data_c + train_data_cel

In [23]:
len(train_data)

2088

In [25]:
train_data[0][0].shape

torch.Size([3, 224, 224])

test data

In [26]:
# select the path
target_path = "/content/drive/MyDrive/DLpj/test_target_2_/*.*"
test_data_charm_zu = []
for file in glob.glob(target_path):
  image_read = cv2.imread(file)
  # resize_img = cv2.resize(image_read, (224, 224))
  test_data_charm_zu.append(image_read)
  # # wait for 1 second
  # k = cv2.waitKey(1000)
  # # destroy the window
  # cv2.destroyAllWindows()


In [27]:
len(test_data_charm_zu)

346

In [28]:
test_data_c = []
for img in test_data_charm_zu:
  img = data_transform(img)
  test_data_c.append((img, 1))

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


In [29]:
len(test_data_c)

346

In [30]:
test_data_c = random.sample(test_data_c, 261)

In [31]:
len(test_data_c)

261

In [32]:
# select the path
target_path = "/content/drive/MyDrive/DLpj/test_celebrities/*.*"
test_data_celebrities = []
for file in glob.glob(target_path):
  image_read = cv2.imread(file)
  # resize_img = cv2.resize(image_read, (224, 224))
  test_data_celebrities.append(image_read)
  # # wait for 1 second
  # k = cv2.waitKey(1000)
  # # destroy the window
  # cv2.destroyAllWindows()


In [33]:
len(test_data_celebrities)

261

In [34]:
test_data_cel = []
for img in test_data_celebrities:
  img = data_transform(img)
  test_data_cel.append((img, 0))

In [35]:
len(test_data_cel)

261

In [36]:
test_data = test_data_c + test_data_cel

In [37]:
len(test_data)

522

In [42]:
#check
# select the path
target_path = "/content/drive/MyDrive/DLpj/check_images_2/*.*"
check_ = []
for file in glob.glob(target_path):
  image_read = cv2.imread(file)
  resize_img = cv2.resize(image_read, (224, 224))
  check_.append(resize_img)
  # # wait for 1 second
  # k = cv2.waitKey(1000)
  # # destroy the window
  # cv2.destroyAllWindows()

check = []
for img in check_:
  img = data_transform(img)
  check.append(img)


check = torch.stack(check, dim=0)

check = check.to(device)

**Model setting**

freeze

In [38]:
resnet = InceptionResnetV1(pretrained='vggface2', device=device).eval()

for param in resnet.parameters():
    param.requires_grad = False

  0%|          | 0.00/107M [00:00<?, ?B/s]

In [39]:
#defining the network

class Tuning(nn.Module):

  def __init__(self):
    super(Tuning,self).__init__()
    self.classifier = nn.Sequential(
            nn.Linear(512, 128),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(128, 2),
            nn.Softmax(dim=1)

    )

    
    # self.classifier = nn.Sequential(
    #         nn.Linear(512, 2),
    #         nn.Softmax(dim=1)
          
    # )


  def forward(self,x):
    x = resnet(x)
    x = self.classifier(x)
    return x

model = Tuning().to(device) ## GPU 사용

In [40]:
batch_size = 128  # 한번에 할 연산량


train_loader = DataLoader(dataset=train_data,
                          batch_size=batch_size,
                          shuffle=True)

test_loader = DataLoader(dataset=test_data,
                         batch_size=batch_size,
                         shuffle=True)

dataloaders = {"train": train_loader, "val": test_loader}

dataset_sizes = {"train": len(train_data), "val": len(test_data)}


In [41]:
criterion = nn.CrossEntropyLoss()
# Observe that all parameters are being optimized
optimizer_ft =  optim.SGD(model.parameters(), lr=1.5, momentum=0.5)
# Decay LR by a factor of *gamma* every *step_size* epochs
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer_ft, step_size=30, gamma=0.1)

# # Observe that all parameters are being optimized
# optimizer_ft =  optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
# # Decay LR by a factor of *gamma* every *step_size* epochs
# exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [42]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()
    FT_losses = []
    T_acc = []
    V_acc = []
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

    # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode
            running_loss = 0.0
            running_corrects = 0
            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                # zero the parameter gradients
                optimizer.zero_grad()
                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
           
                    loss = criterion(outputs, labels)
                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        scheduler.step()
      
                FT_losses.append(loss.item())
                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() /dataset_sizes[phase]
            
            if phase == 'train':
              T_acc.append(epoch_acc)
            else:
              V_acc.append(epoch_acc)

            print('{} Loss: {:.6f} Acc: {:.6f}'.format(phase, epoch_loss, epoch_acc))
            
            
            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())


    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    
    T_acc = torch.tensor(T_acc)
    V_acc = torch.tensor(V_acc)

    T_acc.detach().cpu()
    V_acc.detach().cpu()

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, FT_losses, T_acc, V_acc

In [ ]:
model_ft, FT_losses, T_acc, V_acc = train_model(model, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=100)

plt.figure(figsize=(10,5))
plt.title("FRT Loss During Training")
plt.plot(FT_losses, label="FT loss")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()

plt.figure(figsize=(10,5))
plt.title("Changes in accuracy by epochs")
plt.plot(T_acc, label="train_acc")
plt.plot(V_acc, label="test_acc")
plt.xlabel("epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

In [ ]:
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

# 옵티마이저의 state_dict 출력
print("Optimizer's state_dict:")
for var_name in optimizer_ft.state_dict():
    print(var_name, "\t", optimizer_ft.state_dict()[var_name])

In [45]:
torch.save(model.state_dict(), "/content/drive/MyDrive/DLpj/model_for_inference/model_v2.pt")

Unfreeze

In [ ]:
model_ft = InceptionResnetV1(pretrained='vggface2', classify= False, num_classes =2 ,device=device)

In [ ]:
list(model_ft.children())[-6:]
layer_list = list(model_ft.children())[-5:] # all final layers
model_ft = nn.Sequential(*list(model_ft.children())[:-5])
for param in model_ft.parameters():
    param.requires_grad = False

class Flatten(nn.Module):
    def __init__(self):
        super(Flatten, self).__init__()
        
    def forward(self, x):
        x = x.view(x.size(0), -1)
        return x
class normalize(nn.Module):
    def __init__(self):
        super(normalize, self).__init__()
        
    def forward(self, x):
        x = F.normalize(x, p=2, dim=1)
        return x

In [ ]:
model_ft.avgpool_1a = nn.AdaptiveAvgPool2d(output_size=1)
model_ft.last_linear = nn.Sequential(
    Flatten(),
    nn.Linear(in_features=1792, out_features=512, bias=False),
    # nn.ReLU(inplace=True),
    # nn.Dropout(),
    # nn.Linear(512, 128, bias=False),
    normalize()
)
model_ft.logits = nn.Linear(512, 2)
model_ft.softmax = nn.Softmax(dim=1)
model_ft = model_ft.to(device)

In [ ]:
learning_rate = 0.5
# criterion = nn.BCELoss()
criterion = nn.CrossEntropyLoss()
# Observe that all parameters are being optimized
optimizer_ft =  optim.SGD(model.parameters(), lr=learning_rate, momentum=0.5)
# Decay LR by a factor of *gamma* every *step_size* epochs
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer_ft, step_size=30, gamma=0.1)

# # Observe that all parameters are being optimized
# optimizer_ft =  optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
# # Decay LR by a factor of *gamma* every *step_size* epochs
# exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [ ]:
model_ft, FT_losses = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=100)

plt.figure(figsize=(10,5))
plt.title("FRT Loss During Training")
plt.plot(FT_losses, label="FT loss")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()

Epoch 0/99
----------
train Loss: 0.695366 Acc: 0.443487
val Loss: 0.694180 Acc: 0.452107
Epoch 1/99
----------
train Loss: 0.695348 Acc: 0.432471
val Loss: 0.694805 Acc: 0.440613
Epoch 2/99
----------
train Loss: 0.695355 Acc: 0.437739
val Loss: 0.694963 Acc: 0.432950
Epoch 3/99
----------
train Loss: 0.695314 Acc: 0.442529
val Loss: 0.694906 Acc: 0.434866
Epoch 4/99
----------
train Loss: 0.695320 Acc: 0.443487
val Loss: 0.694994 Acc: 0.432950
Epoch 5/99
----------
train Loss: 0.695411 Acc: 0.442529
val Loss: 0.694971 Acc: 0.434866
Epoch 6/99
----------
train Loss: 0.695278 Acc: 0.438697
val Loss: 0.695020 Acc: 0.434866
Epoch 7/99
----------
train Loss: 0.695355 Acc: 0.431513
val Loss: 0.694965 Acc: 0.434866
Epoch 8/99
----------
train Loss: 0.695319 Acc: 0.431513
val Loss: 0.694969 Acc: 0.434866
Epoch 9/99
----------
train Loss: 0.695262 Acc: 0.439655
val Loss: 0.694937 Acc: 0.434866
Epoch 10/99
----------
train Loss: 0.695331 Acc: 0.440613
val Loss: 0.694900 Acc: 0.434866
Epoch 11/

KeyboardInterrupt: ignored